This is my most up to date and as annotated as possible script. For the data set, have it in you local directory (It's uploaded to github too)


Update 5/6
----------

Training and testing on past 5 seasons data

In [23]:
#import dependences

import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  
from sklearn import preprocessing, cross_validation
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense
np.random.seed(7) #why random?


/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


002yy0--- games from 0001 to 1230



In [52]:
from nba_py import game

In [68]:
game.BoxscoreSummary('0021300001').game_info()

,GAME_DATE,ATTENDANCE,GAME_TIME
0,"TUESDAY, OCTOBER 29, 2013",18165,2:18


In [82]:
GAME = game.Boxscore(game_id='0041700117')

In [83]:
GAME.team_stats()

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0041700117,1610612749,Bucks,MIL,Milwaukee,240:00,35,74,0.473,9,27,0.333,17,18,0.944,4,28,32,21,4,3,15,20,96,-16.0
1,0041700117,1610612738,Celtics,BOS,Boston,240:00,45,84,0.536,9,26,0.346,13,21,0.619,11,31,42,28,9,4,14,23,112,16.0


In [45]:
exgame = game.BoxscoreMisc(game_id='0021701217')

In [25]:
exgame.sql_team_misc()

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,PTS_OFF_TOV,PTS_2ND_CHANCE,PTS_FB,PTS_PAINT,OPP_PTS_OFF_TOV,OPP_PTS_2ND_CHANCE,OPP_PTS_FB,OPP_PTS_PAINT,BLK,BLKA,PF,PFD
0,0021701217,1610612762,Jazz,UTA,Utah,240:00,25.0,15.0,19.0,62.0,16.0,8.0,6.0,38.0,6,3,20,18
1,0021701217,1610612744,Warriors,GSW,Golden State,240:00,16.0,8.0,6.0,38.0,25.0,15.0,19.0,62.0,3,6,18,20


In [26]:
exgame = game.Boxscore(game_id='0021701217')

In [27]:
exgame.team_stats()

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0021701217,1610612762,Jazz,UTA,Utah,240:00,48,90,0.533,13,35,0.371,10,13,0.769,12,42,54,25,10,6,15,20,119,40.0
1,0021701217,1610612744,Warriors,GSW,Golden State,240:00,30,86,0.349,5,19,0.263,14,16,0.875,9,30,39,14,7,3,15,18,79,-40.0


fg% 3p% blocks tos assists drbs orbs trbs 
?   ?    y      y                      y 

In [3]:
def make_network(FILENAME,sklearn=False,keras=False,normalize=True):
    from pandas import read_csv,get_dummies
    import numpy as np
    from sklearn import cross_validation
    from sklearn.neural_network import MLPClassifier
    
    """
    Given the csv input of all the box scores, arrange it such that the home and away teams are lined up, 
    unnecessary columns removed, and hot encoding is done. Other stuff too probably. Such as normalization, but I 
    didn't do that!
    
    Note that this data has already been doctored from its original form, taking out most unnecessary columns but
    those could be useful later on.
    
    
    Parameters
    ----------
    FILENAME : file
        The csv of the data.
        
    Returns
    -------
    
    
    model : object
        MLP which can predict the outcome of NBA games
        
    """
    #Read in file, remove attempted and # and only account for % since that's more predictive in nature. 
    #*retrospectively that doesn't make sense, could be worth changing!
    data = read_csv(FILENAME) 
    data['3P%'] = np.divide(data['3P'].values,data['3PA'].values) 
    del data['3P'],data['3PA']
    data['FG%'] = np.divide(data['FG'].values,data['FGA'].values)
    del data['FG'],data['FGA']
    data['FT%'] = np.divide(data['FT'].values,data['FTA'].values)
    del data['FT'],data['FTA']
    data = get_dummies(data)
    del data['VENUE_Home'],data['VENUE_Road']
    #print(data)
    

    dat = []
    
    #reshape the dataset so now each colummn has roadstats and homestats concatenated into the same row, used for NN 
    
    for i in range(len(data.values)):
        data.values[i] = np.reshape(data.values[i],newshape=[1,len(data.values[i])])
    for p in range(int(len(data.values)/2)):
        fullboxgame = np.concatenate((data.values[2*p],data.values[(2*p)+1]))
        dat.append(fullboxgame)
    
    #convert list to array, now possible to array operations previously not possible
    dat = np.array(dat)   
    
    #now to find out the score of the game, and whether or not the spread was covered. 
    roadpts = dat[:,8] #column of all the points scored by road team 
    homepts = dat[:,21] #vice versa of above
    #print("roadpts",roadpts)
    endspreadS = roadpts-homepts  #all the final spreads of the game
    openingspreadS = dat[:,9]  #what the predicted spread of ther game was. 
    y = []
    
    #Now I iterated over all these, and hot encoded the labels of each to see whether or not the spread was covered
    #and by what team. 
    for j in range(len(endspreadS)):  
        openspread = openingspreadS[j]
       # print("this is the spread of the road team " + str(openspread))
        endspread = endspreadS[j]
       # print("the road team won by  .. " + str(endspread))
       # if endspread>openspread:
        #    y.append(np.array([0,1,0]))  #OK, now make sure this is formateed properly!
        if openspread + endspread <0:
            y.append(np.array([0,1,0]))  #home team covered
        elif openspread + endspread >0:
            y.append(np.array([1,0,0]))  #road covered
        else: 
            y.append(np.array([0,0,1]))  #push!


        
    y = np.array(y)  #same explanation as above
    X1 = np.concatenate((dat[:,0:8],dat[:,10:21]),axis=1)  #reshaping arrays,
    #since everything got out of order I have to mash it together myself. 
    X = np.concatenate((X1,dat[:,23:26]),axis=1)    #need to go one further column to snag HFT% 
    if normalize:
        
        scaler = MinMaxScaler()
        MinMaxScaler(copy=True, feature_range=(0, 1))

        scaler.fit(X)
        X = scaler.transform(X)

    X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.27)
    #print((X[0]))
    #print(np.shape(X[0]))

    #now to construct a model 
    if sklearn: 
        model = MLPClassifier()
        model.shuffle = True
        model.batch_size = 25
    #model.n_layers_ = 1000000
    #model.n_outputs_= 1000000
    #These don't do anything, have to adjust the layers in some different way! Keras is useful for this.
        model.fit(X_train,y_train)
        print(model.score(X_test,y_test))
    if keras:
        print("keras NN goes here")
        model=Sequential()
        model.add(Dense(22,input_dim=np.shape(X)[1],activation='relu'))
        model.add(Dense(30,activation='relu'))
        model.add(Dense(50,activation='relu'))
        model.add(Dense(30,activation='relu'))
        model.add(Dense(22,activation='relu'))

        model.add(Dense(3,activation='sigmoid'))

        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        model.fit(X_train,y_train,batch_size=40,epochs=20,validation_split=.2)
        scores = model.evaluate(X_test,y_test)
        print(scores[1])

    
    
    return model,scaler
    

The next function I have is for turning the current nba team statistics (either over the entire season or some stretch) into an array of the same shape and information as the one used for the box scores above. 

In [4]:
def make_prediction_data(filename):
    from pandas import read_csv,get_dummies
    import numpy as np
    from sklearn import cross_validation
    data = read_csv(filename)
    #convert to per game stats and sort columns. 
    data['ORB'] =  np.divide(data['ORB'].values,data['G'].values)
    data['DRB'] =  np.divide(data['DRB'].values,data['G'].values)
    data['TRB'] =  np.divide(data['TRB'].values,data['G'].values)
    data['AST'] =  np.divide(data['AST'].values,data['G'].values)
    data['STL'] =  np.divide(data['STL'].values,data['G'].values)
    data['BLK'] =  np.divide(data['BLK'].values,data['G'].values)
    data['TOV'] =  np.divide(data['TOV'].values,data['G'].values)
    data['PF'] =  np.divide(data['PF'].values,data['G'].values)
    teams  = data['Team']

    data = data[['ORB' , 'DRB'  ,'TRB' ,  'AST' , 'PF' , 'STL' , 'TOV' , 'BLK' ,'3P%','FG%' ,'FT%']]
   # print("Here is every teams index value: ")
   # print(teams)
    return teams,data

In [5]:
def game_maker(roadteam,hometeam,scaler):
    """After creating a properly formated table, this concats the desired teams so they can be predicted. 
        Based on get team index # based on output of predictor, and make it the input for stats ie GSW are stats[0].
        and so on!
    """
    game = np.concatenate((roadteam,hometeam))
    game = [game]
    game = scaler.transform(game)
    return game

In [6]:
filetouse = '1718nbateamstats.csv'  #downloaded from basketball reference, and specified date 

teams, stats =make_prediction_data(filetouse)

Playoff teams + Index #
----------------------
Warriors 0
-----------
Rockets 1
-----------
Pelicans 2
-----------
Raptors 3 
-----------
Cavs 4
-----------
76ers 6
-----------
Timberwolves 7
-----------
Thunder 11
-----------
Wizards 12
-----------
Bucks 14
-----------
Trail Blazers 15
-----------
Pacers 16
-----------
Jazz 18
-----------
Celtics 19
-----------
Heat 22
-----------
Spurs 26
-----------

Now or tomorrow it would be smark to analyze all the different games, a ton have already been played, could be worth investigating what my playoff record is after round 1 for a specific series


--Some things worth noting are playoff injuries, etc. like how the sixers are never really at their full strength and same goes for warriors but whatever

In [9]:
#returns the keras model for prediction, along with the scaling tool to normalize future data
nbapredictor,scaler = make_network('1517_boxscores.csv',sklearn=False,keras=True,normalize=True)

keras NN goes here
Train on 1532 samples, validate on 384 samples
Epoch 1/20
1532/1532 [==============================] - 1s 643us/step - loss: 0.8588 - acc: 0.5020 - val_loss: 0.7679 - val_acc: 0.4687
Epoch 2/20
1532/1532 [==============================] - 0s 47us/step - loss: 0.7850 - acc: 0.5692 - val_loss: 0.7635 - val_acc: 0.4818
Epoch 3/20
1532/1532 [==============================] - 0s 50us/step - loss: 0.7735 - acc: 0.5849 - val_loss: 0.7397 - val_acc: 0.6302
Epoch 4/20
1532/1532 [==============================] - 0s 53us/step - loss: 0.7344 - acc: 0.6782 - val_loss: 0.6471 - val_acc: 0.7474
Epoch 5/20
1532/1532 [==============================] - 0s 42us/step - loss: 0.6371 - acc: 0.7219 - val_loss: 0.6159 - val_acc: 0.7057
Epoch 6/20
1532/1532 [==============================] - 0s 46us/step - loss: 0.5617 - acc: 0.7644 - val_loss: 0.5059 - val_acc: 0.7943
Epoch 7/20
1532/1532 [==============================] - 0s 50us/step - loss: 0.5414 - acc: 0.7813 - val_loss: 0.4875 - val_

In [12]:
nbapredictor.predict(bosphi)

array([[1, 0, 0]])

In [13]:
#west round 2
utahou = game_maker(stats.values[18],stats.values[1],scaler) #minny at houston
houuta = game_maker(stats.values[1],stats.values[18],scaler) #vice versa
nopgsw = game_maker(stats.values[2],stats.values[0],scaler)  # pelicans at gsw
gswnop = game_maker(stats.values[0],stats.values[2],scaler)


In [17]:
nbapredictor.predict(gswnop)

array([[1, 0, 0]])

In [ ]:
#western conference round 1
sasgsw = game_maker(stats.values[26],stats.values[0],scaler)
gswsas = game_maker(stats.values[0],stats.values[26],scaler)
minhou = game_maker(stats.values[7],stats.values[1],scaler)
houmin = game_maker(stats.values[1],stats.values[7],scaler)
utaokc = game_maker(stats.values[18],stats.values[11],scaler)
okcuta = game_maker(stats.values[11],stats.values[18],scaler)
noppor = game_maker(stats.values[2],stats.values[15],scaler)
pornop = game_maker(stats.values[15],stats.values[2],scaler)


In [ ]:
nbapredictor.predict(indcle)

In [ ]:
#okc uta too close in this version, gonna re run 

In [ ]:
#eastern conference round 1
miaphi = game_maker(stats.values[22],stats.values[6],scaler)
phimia = game_maker(stats.values[6],stats.values[22],scaler)
milbos = game_maker(stats.values[14],stats.values[19],scaler)
bosmil = game_maker(stats.values[19],stats.values[14],scaler)
wastor = game_maker(stats.values[12],stats.values[3],scaler)
torwas = game_maker(stats.values[3],stats.values[12],scaler)

indcle = game_maker(stats.values[16],stats.values[4],scaler)
cleind = game_maker(stats.values[4],stats.values[16],scaler)

In [ ]:
nbapredictor.predict(torwas)

In [ ]:
#Round2matchups
phibos = game_maker(stats.values[6],stats.values[19],scaler) #philly at boston
bosphi = game_maker(stats.values[19],stats.values[6],scaler) #vice versa
cletor = game_maker(stats.values[4],stats.values[3],scaler)  #cleveland at toronto
torcle = game_maker(stats.values[3],stats.values[4],scaler) #vice versa
utahou = game_maker(stats.values[18],stats.values[1],scaler) #minny at houston
houuta = game_maker(stats.values[1],stats.values[18],scaler) #vice versa
nopgsw = game_maker(stats.values[2],stats.values[0],scaler)  # pelicans at gsw
gswnop = game_maker(stats.values[0],stats.values[2],scaler)

gswsac = game_maker(stats.values[29],stats.values[0],scaler)


In [ ]:
#so who covers the various game 1s? 
nbapredictor.predict(cletor)

In [ ]:
nbapredictor.predict(nopgsw)

In [ ]:
0.8464921 + 0.45634875 + 0.10764076

In [ ]:
0.9274949 + 0.3562849 + 0.10068643

In [ ]:
minhou = game_maker(stats.values[7],stats.values[1]) #minny at houston
houmin = game_maker(stats.values[1],stats.values[7]) #vice versa

game1 = game_maker(stats.values[1],stats.values[18])
game2 = game_maker(stats.values[3],stats.values[15])
game3 = game_maker(stats.values[7],stats.values[13])

In [ ]:
nbapredictor.predict(np.array([game1,game2,game3]))